<h1 style="color:black" align="center">Интерпретация моделей</h1>

<h1 style="color:#008B8B">1. Что находят свёрточные сети?</h1>

https://cs.nyu.edu/~fergus/papers/zeilerECCV2014.pdf

* Можно для каждого фильтра найти кусочки картинок, дающие самый сильный отклик
* Сделаем это для AlexNet

Можно среди всей обучающей выборки найти картинку и кусочек на ней, где фильтр дает наибольший отклик. Посомтрим на пример 3-го сверточного слоя:

<img src='img/lecture07/1.png'>

## Максимизация вероятности класса

### Вариант 1

https://arxiv.org/pdf/1312.6034.pdf

Возьмем обученную нейронную сеть $a_y(x)$, где $y$ - это выход конкретного класса, например, вероятность того, что это кошка. Дальше подаем на вход случайную картинку $x$, после чего начинаем изменять данную картинку градиентным спуском, чтобы максимизировать вероятность того, что на картинке кошка. Мы же можем градиентным спуском обучать не параметры, а саму картинку. Тогда посчитам backprop выходы сетки по пикселям входной картинки, после чего сдвигать пиксели, чтобы вероятность максимизировалась.

Так же, имеется регуляризатор, который ограничивает величины интенсивности цветов, которые записаны в пикселях.

$\large a_y(x) - \lambda \|x\|_2^2 \to \underset{x}{\text{min}}$

Посмотрим, какие картинки с точки зрения нейронной сети с большей вероятностью содержат тот или иной класс:

<img src='img/lecture07/2.png'>

### Вариант 2

Здесь улучшенная версия

https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html

<img src='img/lecture07/3.png'>

Давайте теперь инициализировть вход не случайным шумом, как это было на прошлых картинках, а возьмем за основу некоторую картинку и будем максимизировать вероятность некоторого класса.

<img src='img/lecture07/4.png'>

Дальше можно взять большие картинки и максимизировать вероятность некоторого класса.

<img src='img/lecture07/5.png'>